## PyTorch数据读入是通过Dataset+DataLoader的方式完成的，Dataset定义好数据的格式和数据变换形式，DataLoader用iterative的方式不断读入批次数据。

# Dataset

自定义一个继承 Dataset类的类 ，需要重写以下三个函数：

__init__：传入数据，或者像下面一样直接在函数里加载数据；

__len__：返回这个数据集一共有多少个item；

__getitem__:返回一条训练数据，并将其转换成tensor。

通常还会在其中增加一个collate_fn函数，用于DataLoader，使用这个参数可以自己操作每个batch的数据，比如说在自然语言处理的命名实体识别任务中，在该函数中对每个batch中的样本都padding到同一长度等。

In [3]:
import torch
from torch.utils.data import Dataset

class SquareDataset(Dataset):
    def __init__(self, numbers, transform=None):
        self.numbers = numbers
        self.labels = [x**2 for x in numbers]
        self.transform = transform

    def __len__(self):
        return len(self.numbers)

    def __getitem__(self, idx):
        sample = self.numbers[idx]
        label = self.labels[idx]
        if self.transform:
            sample = self.transform(sample)
        return sample, label

# 创建数据集
numbers = [1, 2, 3, 4, 5]
dataset = SquareDataset(numbers)

# 使用 DataLoader 加载数据
from torch.utils.data import DataLoader
dataloader = DataLoader(dataset, batch_size=2, shuffle=True)

# 遍历 DataLoader
for batch in dataloader:
    print(batch)

[tensor([1, 2]), tensor([1, 4])]
[tensor([5, 3]), tensor([25,  9])]
[tensor([4]), tensor([16])]


transform 是一个可选参数，通常用于对数据进行预处理或增强。你可以使用 PyTorch 提供的 torchvision.transforms 模块中的函数，或者自定义转换函数。

In [5]:
from torchvision import transforms

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

dataset = SquareDataset(numbers, transform=transform)

# TensorDataset

torch.utils.data.TensorDataset 是 PyTorch 提供的一个便捷类，用于将多个张量（Tensor）组合成一个数据集。它继承自 torch.utils.data.Dataset，并自动实现了 ___len__ 和 ___getitem__ 方法，因此你无需手动实现这些方法。

TensorDataset 的主要用途是将输入数据和标签数据（或其他相关数据）打包成一个数据集，方便后续通过 DataLoader 进行批量加载。

In [8]:
import torch
from torch.utils.data import TensorDataset

# 创建输入数据和标签
X = torch.tensor([[1, 2], [3, 4], [5, 6]], dtype=torch.float32)  # 3个样本，每个样本2个特征
y = torch.tensor([0, 1, 0], dtype=torch.float32)  # 3个标签

# 创建 TensorDataset
dataset = TensorDataset(X, y)

# 查看数据集大小
print(len(dataset))  # 输出: 3

# 获取单个样本
print(dataset[0])  # 输出: (tensor([1., 2.]), tensor(0.))

3
(tensor([1., 2.]), tensor(0.))


TensorDataset 通常与 DataLoader 结合使用，以便批量加载数据。

In [ ]:
from torch.utils.data import DataLoader

# 创建 DataLoader
dataloader = DataLoader(dataset, batch_size=2, shuffle=True)

# 遍历 DataLoader4
for batch_X, batch_y in dataloader:
    print("Batch X:", batch_X)
    print("Batch y:", batch_y)

Batch X: tensor([[1., 2.],
        [3., 4.]])
Batch y: tensor([0., 1.])
Batch X: tensor([[5., 6.]])
Batch y: tensor([0.])


TensorDataset 可以接受多个张量，例如输入数据、标签数据和其他辅助数据。

In [ ]:
# 创建输入数据、标签和其他辅助数据
X = torch.tensor([[1, 2], [3, 4], [5, 6]], dtype=torch.float32)
y = torch.tensor([0, 1, 0], dtype=torch.float32)
z = torch.tensor([10, 20, 30], dtype=torch.float32)  # 辅助数据

# 创建 TensorDataset
dataset = TensorDataset(X, y, z)

# 获取单个样本
print(dataset[1])  # 输出: (tensor([3., 4.]), tensor(1.), tensor(20.))

(tensor([3., 4.]), tensor(1.), tensor(20.))


# DataLoader

DataLoader包括三个参数：

dataset：传入的数据；

shuffle = True:是否打乱数据；

collate_fn函数：使用这个参数可以自己操作每个batch的数据。

drop_last：告诉如何处理划分batch后剩下的最后不足一个batch的样本集合，True就抛弃，否则保留。